# MNIST BDP V2

Now that we have the first version done, we are going to enrich our components even further. First, we load the previous JSON.

In [1]:
import sys
sys.path.append("../../pybdp")
from src import pybdp
#import pybdp
from pprint import pprint

# Start with an empty project
project = pybdp.load_from_json("../JSON/Supervised Learning V1.json")
print(project)

< Project
Toolbox:

< Toolbox
Blocks: ['Experiment', 'Load Supervised Features', 'Supervised Learning']
Spaces: ['Model', 'Evaluation Metrics', 'X Train', 'Y Train', 'X Test', 'Y Test'] >

Workbench:

<Workbench
Processors: ['MNIST Experiment', 'Load MNIST', 'Default Supervised Learning']
Wires: ['W1', 'W2', 'W3', 'W4']
Systems: ['MNIST Experiment System'] > >


## Load MNIST

Currently "Load MNIST" is a primitive processor, but we are going to expand it to be a composite processor entailing a few specific things:

1. Loading the dataset
2. Conducting a test-train split
3. Doing image normalization for training data
4. Doing image normalization for testing data

In [2]:
# Add spaces
project.add_space(id="X",
                  name="X",
                  description="X Data")

project.add_space(id="Y",
                  name="Y",
                  description="Y Data")

# Add blocks
project.add_block(id="Load Supervised Dataset",
                  name="Load Supervised Dataset",
                  description="Loads the data for a supervised learning problem",
                  domain=[],
                  codomain=["X", "Y"])

project.add_block(id="Cross Validation Split",
                  name="Cross Validation Split",
                  description="Splits data into training and testing sets",
                  domain=["X", "Y"],
                  codomain=["X Train", "Y Train", "X Test", "Y Test"])

project.add_block(id="Training Data Preprocessing",
                  name="Training Data Preprocessing",
                  description="Preprocesses training data for model training",
                  domain=["X Train", "Y Train"],
                  codomain=["X Train", "Y Train"])

project.add_block(id="Testing Data Preprocessing",
                  name="Testing Data Preprocessing",
                  description="Preprocesses testing data for model evaluation, optionally can use the training data in addition",
                  domain=["X Train", "Y Train", "X Test", "Y Test"],
                  codomain=["X Test", "Y Test"])

# Add processors
project.add_processor(id="Load MNIST Dataset",
                      name="Load MNIST Dataset",
                      description="Loads the MNIST dataset",
                      parent_id="Load Supervised Dataset")

project.add_processor(id="Test-Train Split",
                      name="Test-Train Split",
                      description="Splits data into one training and one test set",
                      parent_id="Cross Validation Split")

project.add_processor(id="Image Normalization Preprocessing - Training",
                      name="Image Normalization Preprocessing - Training",
                      description="Preprocesses image-based training data",
                      parent_id="Training Data Preprocessing")

project.add_processor(id="Image Normalization Preprocessing - Testing",
                      name="Image Normalization Preprocessing - Testing",
                      description="Preprocesses testing data for model evaluation",
                      parent_id="Testing Data Preprocessing",)

### Wires & System

Similar to before, we find the wires, then build and attach the system. For the wires, we end up using all the possible wires because it is all of the ones we need and nothing more.

In [3]:
for p1, p2 in zip(["Load MNIST Dataset", "Test-Train Split", "Test-Train Split"], ["Test-Train Split", "Image Normalization Preprocessing - Training", "Image Normalization Preprocessing - Testing"]):
    print("Potential wires between processors: {} and {}".format(p1, p2))
    pprint(project.processors_map[p1].find_potential_wires(project.processors_map[p2]))
    print()
    print()

Potential wires between processors: Load MNIST Dataset and Test-Train Split
{'Ports': [],
 'Terminals': [{'Parent': 'X',
                'Source': {'Index': 0, 'Processor': 'Load MNIST Dataset'},
                'Target': {'Index': 0, 'Processor': 'Test-Train Split'}},
               {'Parent': 'Y',
                'Source': {'Index': 1, 'Processor': 'Load MNIST Dataset'},
                'Target': {'Index': 1, 'Processor': 'Test-Train Split'}}]}


Potential wires between processors: Test-Train Split and Image Normalization Preprocessing - Training
{'Ports': [],
 'Terminals': [{'Parent': 'X Train',
                'Source': {'Index': 0, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 0,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Training'}},
               {'Parent': 'Y Train',
                'Source': {'Index': 1, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 1,


In [4]:
project.add_wires([{'Parent': 'X',
                'Source': {'Index': 0, 'Processor': 'Load MNIST Dataset'},
                'Target': {'Index': 0, 'Processor': 'Test-Train Split'}},
               {'Parent': 'Y',
                'Source': {'Index': 1, 'Processor': 'Load MNIST Dataset'},
                'Target': {'Index': 1, 'Processor': 'Test-Train Split'}},
                {'Parent': 'X Train',
                'Source': {'Index': 0, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 0,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Training'}},
               {'Parent': 'Y Train',
                'Source': {'Index': 1, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 1,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Training'}},
                {'Parent': 'X Train',
                'Source': {'Index': 0, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 0,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Testing'}},
               {'Parent': 'Y Train',
                'Source': {'Index': 1, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 1,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Testing'}},
               {'Parent': 'X Test',
                'Source': {'Index': 2, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 2,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Testing'}},
               {'Parent': 'Y Test',
                'Source': {'Index': 3, 'Processor': 'Test-Train Split'},
                'Target': {'Index': 3,
                           'Processor': 'Image Normalization Preprocessing - '
                                        'Testing'}}],
                auto_increment=True)

pprint(project.wires)

[< Wire ID: W1 Space: X Train Source: (Load MNIST, 0) Target: (Default Supervised Learning, 1) >,
 < Wire ID: W2 Space: Y Train Source: (Load MNIST, 1) Target: (Default Supervised Learning, 2) >,
 < Wire ID: W3 Space: X Test Source: (Load MNIST, 2) Target: (Default Supervised Learning, 3) >,
 < Wire ID: W4 Space: Y Test Source: (Load MNIST, 3) Target: (Default Supervised Learning, 4) >,
 < Wire ID: W5 Space: X Source: (Load MNIST Dataset, 0) Target: (Test-Train Split, 0) >,
 < Wire ID: W6 Space: Y Source: (Load MNIST Dataset, 1) Target: (Test-Train Split, 1) >,
 < Wire ID: W7 Space: X Train Source: (Test-Train Split, 0) Target: (Image Normalization Preprocessing - Training, 0) >,
 < Wire ID: W8 Space: Y Train Source: (Test-Train Split, 1) Target: (Image Normalization Preprocessing - Training, 1) >,
 < Wire ID: W9 Space: X Train Source: (Test-Train Split, 0) Target: (Image Normalization Preprocessing - Testing, 0) >,
 < Wire ID: W10 Space: Y Train Source: (Test-Train Split, 1) Target: (

In [5]:
      {
        "ID": "Load MNIST System",
        "Name": "Load MNIST System",
        "Description": "The system representing loading the MNIST dataset, preparing the cross validation split, and pre-processing the data",
        "Processors": [
          "Load MNIST Dataset",
          "Test-Train Split",
          "Image Normalization Preprocessing - Training",
          "Image Normalization Preprocessing - Testing"
        ],
        "Wires": ["W1", "W2", "W3", "W4", "W5", "W6", "W7", "W8"]
      },

({'ID': 'Load MNIST System',
  'Name': 'Load MNIST System',
  'Description': 'The system representing loading the MNIST dataset, preparing the cross validation split, and pre-processing the data',
  'Processors': ['Load MNIST Dataset',
   'Test-Train Split',
   'Image Normalization Preprocessing - Training',
   'Image Normalization Preprocessing - Testing'],
  'Wires': ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8']},)

In [6]:
{
        "Description": "Load MNIST",
        "ID": "Load MNIST",
        "Name": "Load MNIST",
        "Parent": "Load Supervised Features",
        "Ports": [],
        "Subsystem": {
          "Port Mappings": [],
          "System ID": "Load MNIST System",
          "Terminal Mappings": [
            {
              "Index": 0,
              "Processor": "Image Normalization Preprocessing - Training"
            },
            {
              "Index": 1,
              "Processor": "Image Normalization Preprocessing - Training"
            },
            {
              "Index": 0,
              "Processor": "Image Normalization Preprocessing - Testing"
            },
            {
              "Index": 1,
              "Processor": "Image Normalization Preprocessing - Testing"
            }
          ]
        },
        "Terminals": [
          "X Train Array",
          "Y Train Array",
          "X Test Array",
          "Y Test Array"
        ]
      },


{
        "Description": "Conducts supervised learning using the defaults of the model",
        "ID": "Default Supervised Learning",
        "Name": "Default Supervised Learning",
        "Parent": "Supervised Learning",
        "Ports": [
          "Model",
          "X Train Array",
          "Y Train Array",
          "X Test Array",
          "Y Test Array"
        ],
        "Subsystem": {
          "Port Mappings": [
            {
              "Index": 0,
              "Processor": "Fit Supervised Model - Default"
            },
            {
              "Index": 1,
              "Processor": "Fit Supervised Model - Default"
            },
            {
              "Index": 2,
              "Processor": "Fit Supervised Model - Default"
            },
            {
              "Index": 1,
              "Processor": "Evaluate Supervised Model - Default"
            },
            {
              "Index": 2,
              "Processor": "Evaluate Supervised Model - Default"
            }
          ],
          "System ID": "Default Supervised Learning System",
          "Terminal Mappings": [
            {
              "Index": 0,
              "Processor": "No Post-processing"
            }
          ]
        },
        "Terminals": ["Evaluation Metrics"]
      },

({'Description': 'Conducts supervised learning using the defaults of the model',
  'ID': 'Default Supervised Learning',
  'Name': 'Default Supervised Learning',
  'Parent': 'Supervised Learning',
  'Ports': ['Model',
   'X Train Array',
   'Y Train Array',
   'X Test Array',
   'Y Test Array'],
  'Subsystem': {'Port Mappings': [{'Index': 0,
     'Processor': 'Fit Supervised Model - Default'},
    {'Index': 1, 'Processor': 'Fit Supervised Model - Default'},
    {'Index': 2, 'Processor': 'Fit Supervised Model - Default'},
    {'Index': 1, 'Processor': 'Evaluate Supervised Model - Default'},
    {'Index': 2, 'Processor': 'Evaluate Supervised Model - Default'}],
   'System ID': 'Default Supervised Learning System',
   'Terminal Mappings': [{'Index': 0, 'Processor': 'No Post-processing'}]},
  'Terminals': ['Evaluation Metrics']},)